In [ ]:
###paths#####
#############
import sys
import os

pwd = os.getcwd()
modellingpath = pwd.rpartition("modelling")[0] + pwd.rpartition("modelling")[1] 
sys.path.append(modellingpath + '/lib')
#############
# 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle


In [ ]:

cf_matrix_scholes = np.array([[1,0],[0,1]])
ax = sns.heatmap(cf_matrix_scholes, annot=False, cmap='Blues',cbar=False)

ax.set_xlabel('\nNumerical solution')
ax.set_ylabel('Dispersion relation\n');

## Ticket labels - List must be in alphabetical order
ax.yaxis.set_ticklabels(['Turing I','Else'])
ax.xaxis.set_ticklabels(['Pattern','No Pattern'])

## Display the visualization of the Confusion Matrix.
plt.tight_layout()
plt.savefig('cf_scholes')
print('Dispersion relation information')
plt.show()


In [ ]:


cf_matrix_marti = np.array([[1,0,0,0,0],[0,0,0,0,1],[1,0,1,0,0],[0,1,1,0,0],[0,0,0,1,0],[0,0,0,0,1]])
ax = sns.heatmap(cf_matrix_marti, annot=True, cmap='Blues',cbar=False)

ax.set_xlabel('\nNumerical solution')
ax.set_ylabel('Dispersion relation\n');

## Ticket labels - List must be in alphabetical order
ax.yaxis.set_ticklabels(['Turing I','Turing II', 'Turing I Hopf', 'Hopf instability', 'Complex with no instability', 'Real with no instability'], rotation=0)
ax.xaxis.set_ticklabels(['Stationary spatial wave','Oscillatory spatial wave', 'Temportal oscillations', 'Chaotic inhomogeneities', 'Homogeneous'], rotation=90)
## Display the visualization of the Confusion Matrix.
plt.tight_layout()
# plt.savefig('cf_marti')
print('Numerical information')
plt.show()




In [ ]:
circuit_n='turinghill'
variant= 0
n_species=2
mechanism='nogrowth'
L=50; dx =1; J = int(L/dx)
T =3000; dt = 0.05; N = int(T/dt)
boundaryCoeff=2;rate=0.01
filename= lambda mechanism, parID: 'circuit%s_variant%s_bc%s_%s_rate%s_ID%s_L%r_J%r_T%r_N%r'%(circuit_n,variant,boundaryCoeff, mechanism,rate,parID,L,J,T,N)
n_param_sets=2000000
pattern_df = pickle.load(open( modellingpath + '/growth/out/patternAnalysis/%s/%s/pattern/pattern_df_%s.pkl'%(circuit_n,mechanism,filename(mechanism,'x')), 'rb'))


In [ ]:
pattern_df['pattern'].value_counts(normalize=True)
# pattern_df['pattern'].unique()

In [ ]:
pattern_list = ['Temporal Oscillator', 'Non-Stationary heterogeneity','Homogeneous', 'Stationary spatial wave']
# for pattern in lsa_df['pattern'].unique() :
for pattern in pattern_list :
    print(pattern)
    print(pattern_df[pattern_df['pattern']==pattern]['system_class'].value_counts())
    print('-------------------------------------------')
